# Calcul scientifique en Python 

## Les flottants

In [14]:
0.1 ** 100

1.0000000000000056e-100

In [16]:
3602879701896397 * 10 ** 55 // 2 ** 55

1000000000000000055511151231257827021181583404541015625

In [15]:
.1 + .1 + .1 == .3

False

In [17]:
format(0.1, '.17f')

'0.10000000000000001'

## Numpy

### L'objet `np.ndarray`

Types de données, ordre C et ordre F

### `shape` et indexation

### Création de données

Données pré-déterminées et données aléatoires, grilles

## Premier exercice - [Le jeu de la vie de Conway](https://fr.wikipedia.org/wiki/Jeu_de_la_vie)

> Le jeu de la vie n’est pas un jeu, puisqu'il ne nécessite aucun joueur. Il s’agit d’un automate cellulaire, un modèle où chaque état conduit mécaniquement à l’état suivant à partir de règles pré-établies.

> Le « jeu » se déroule sur une grille à deux dimensions, théoriquement infinie (mais de longueur et de largeur finies et plus ou moins grandes dans la pratique), dont les cases — qu’on appelle des « cellules », par analogie avec les cellules vivantes — peuvent prendre deux états distincts : « vivante » ou « morte ».


> À chaque étape, l’évolution d’une cellule est entièrement déterminée par l’état de ses huit voisines de la façon suivante :


* Une cellule morte possédant exactement trois voisines vivantes devient vivante (elle naît).

* Une cellule vivante possédant deux ou trois voisines vivantes le reste, sinon elle meurt.



_Écrivez l'évaluation de la règle pour un pixel_

<button data-toggle="collapse" data-target="#sol" class='btn btn-primary'>Solution</button>
<div id="sol" class="collapse">

```py
import numpy as np
import time

grid_shape = 10, 10
grid = np.random.randint(0, 2, size=grid_shape, dtype=np.uint8)
#grid[1:4, 3] = 1
print(grid)


def eval_pixel(row, col, grid, next_grid):
    """Follows the rules of Conway's game of life to
    flip (or not) a pixel in a grid
    """
    n, m = grid.shape
    sub_grid = grid[
        max(row-1, 0): min(n, row+2),
        max(col-1, 0): min(m, col+2)]
    if grid[row, col] and not (2 < sub_grid.sum() < 5):
        next_grid[row, col] = 0
    elif sub_grid.sum() == 3:
        next_grid[row, col] = 1


def conway_step(grid):
    """Applies Conway's rule for every pixel on a grid
    """
    ii, jj = np.meshgrid(
        np.arange(grid.shape[0]),
        np.arange(grid.shape[1]) )

    next_grid = grid.copy()
    for i, j in zip(ii.ravel(), jj.ravel()):
        eval_neighbors(i, j, grid, next_grid)
    return next_grid


def conway_loop(grid):
    """Play Conways game of life on repeat
    """
    while True:
        try:
            grid = conway_step(grid)
            print(grid)
            time.sleep(1)
            
        except KeyboardInterrupt:
            print('Done')
            break
```

</div>

## Calcul

## Scipy

Exercice 2:

> Créez un signal composé de différentes harmoniques (et de bruit) et évaluez sa FFT.



<button data-toggle="collapse" data-target="#sol2" class='btn btn-primary'>Solution</button>
<div id="sol2" class="collapse">



```python
import numpy as np
import matplotlib.pyplot as plt


t = np.linspace(0, 100, 2**12)
harms = np.random.random(5) * 100
phases = np.random.random(5) * np.pi
signal = np.array(
    [np.sin(omega*t + phi)
      for omega, phi in zip(harms, phases)]
).sum(axis=0)

fft = np.fft.fft(signal)
freqs = np.fft.fftfreq(2**12, 100 / 2**12)

plt.figure()
plt.plot(signal)

plt.figure()
plt.plot(np.abs(freqs), np.abs(fft))

print(np.sort(harms) / (2*np.pi))
```
</div>

In [52]:
signal

array([ 3.06275594,  1.90623805, -0.58831058, ..., -1.7464471 ,
       -1.24271986, -0.28933429])

### Exercice 3:

> Convoluez le précédent signal par un filtre passe bas de fréquence 10 Hz

<button data-toggle="collapse" data-target="#sol3" class='btn btn-primary'>Solution</button>
<div id="sol3" class="collapse">


```python
from scipy.signal import fftconvolve

filtr = np.ones(100)
filtr[10:] = 0

convd = fftconvolve(signal, filtr, mode='same')

plt.plot(t, signal)
plt.plot(t, convd, alpha=0.4)
```

## Pandas 

#### I/O

In [116]:
import pandas as pd
from pathlib import Path

In [118]:
col_file = Path("ifpen_data/SOLUTION3D/ASCII_COLUMN_FILE/sprayA000015_+1.40019e-03.col")

In [119]:
!head {col_file}

1.400185e-03 column format file
               x               y               z          volume         yC12H26
   4.1800000e-02  -5.9999998e-03  -5.9999998e-03   5.1199911e-10   1.0279532e-06
   4.1000001e-02  -5.9999998e-03  -5.9999998e-03   5.1199911e-10   7.7545047e-07
   4.0200000e-02  -5.9999998e-03  -5.9999998e-03   5.1200176e-10   5.9764074e-07
   3.8599999e-02  -5.9999998e-03  -5.2000001e-03   5.1200097e-10   1.0017285e-06
   3.8599999e-02  -5.2000001e-03  -5.9999998e-03   5.1200097e-10   9.5407586e-07
   3.9400000e-02  -5.9999998e-03  -5.9999998e-03   5.1199911e-10   4.5205115e-07
   3.8599999e-02  -5.9999998e-03  -5.9999998e-03   5.1200123e-10   3.0310602e-07
   3.7799999e-02  -5.9999998e-03  -5.2000000e-03   5.1199885e-10   7.0007781e-07


In [174]:
cdf_data.shape[0]**(1/3)

75.59660455248356

In [173]:
602 * 575 * 630

218074500

In [138]:
import ipyvolume as ipv

In [144]:
from matplotlib import cm

In [146]:
cmap = cm.get_cmap("Reds")
normed = (
    cdf_data.yC12H26 - cdf_data.yC12H26.min()) / (
    cdf_data.yC12H26.max() - cdf_data.yC12H26.min())


In [160]:
%pdb

Automatic pdb calling has been turned ON


In [189]:
ipv.clear()
ipv.scatter(
    x=cdf_data.x[cdf_data.y>0].values,
    y=cdf_data.y[cdf_data.y>0].values,
    z=cdf_data.z[cdf_data.y>0].values,
    color=cmap(normed[cdf_data.y>0]),
    size=0.5,
    marker="diamond"
)

ipv.xlim(-1e-2, 1e-2)
ipv.ylim(-1e-2, 1e-2)
ipv.zlim(-1e-2, 1e-2)


ipv.squarelim()

ipv.show()

In [190]:
from scipy.spatial import qhull

In [ ]:
h = plt.hist(cdf_data)